<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/notebooks/Mock_NeighborLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mock-тест load_graph_from_postgres для NeighborLoader v1

Этот тестовый класс охватывает следующие аспекты работы функции load_graph_from_postgres:

- Проверка корректной загрузки структуры графа (узлы и рёбра).
- Реакция на отсутствие рёбер в данных.
- Обработка ошибок подключения или выполнения запросов к БД.
- Совместимость с инструментом NeighborLoader для последующего обучения графовых моделей.

Основные преимущества тестов:

- Исключают зависимость от реальной базы данных за счёт использования моков.
- Гарантируют устойчивость функции к различным сценариям, включая крайние случаи.
- Проверяют, что загруженный граф может быть использован в составе NeighborLoader — критично для задач обучения на основе соседних узлов в графах (GNN).

Небходимо выбрать среду выполнения с GPU: Среда выполнения → Сменить среду выполнения → Графический процессор T4

## 1. Установка зависимостей

In [1]:
install = True
if install:
  # 1. Установка совместимых версий PyTorch и PyG
  !pip install -q torch==2.3.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121

  # 2. Установка зависимостей PyG для CUDA 12.1+ (совместимо с 12.5)
  !pip install -q pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html --no-cache-dir

  # 3. Установка PyTorch Geometric
  !pip install -q torch-geometric==2.5.3

  # 4. Установка psycopg2-binary
  !pip install -q psycopg2-binary

## 2. Полный тест с пояснениями

__Функция ```load_graph_from_postgres```__

В функцию ```load_graph_from_postgres```

* Добавлена обработка исключений вокруг pd.read_sql - Убедились, что даже если SQL-запросы завершаются ошибкой, функция не падает
* Убедились, что conn.close() вызывается в finally - Предотвращаем утечки соединений
* Возвращается Data(...) с пустыми тензорами - Обеспечивается совместимость с моделью, даже если данные не загружены

__Что проверяет каждый тест?__

- ```test_load_graph_returns_correct_data``` - Загрузка данных из БД с корректными узлами и рёбрами
- ```test_load_graph_returns_empty_edges``` - Обработка отсутствия рёбер в БД
- ```test_load_graph_handles_db_errors``` - Обработка ошибок подключения или SQL-запросов

In [2]:
import unittest
from unittest.mock import patch, MagicMock
import pandas as pd
import torch
from torch_geometric.data import Data
import psycopg2
import warnings

# === Тестируемая функция ===
def load_graph_from_postgres(db_config):
    try:
        conn = psycopg2.connect(**db_config)
        try:
            # Загрузка узлов
            query_nodes = "SELECT alias, label, degree, total_received FROM node_attributes"
            nodes_df = pd.read_sql(query_nodes, conn)

            # Загрузка рёбер
            query_edges = "SELECT a, b, total_sent FROM edge_attributes"
            edges_df = pd.read_sql(query_edges, conn)
        finally:
            conn.close()
    except Exception as e:
        # В случае ошибки возвращаем пустой граф
        return Data(
            x=torch.tensor([], dtype=torch.float),
            edge_index=torch.tensor([], dtype=torch.long).view(2, -1),
            edge_attr=torch.tensor([], dtype=torch.float).view(-1, 1),
            y=torch.tensor([], dtype=torch.long)
        )

    # Маппинг alias -> индекс
    alias_to_idx = {alias: idx for idx, alias in enumerate(nodes_df['alias'])}

    # Формирование edge_index и edge_attr
    edges = []
    edge_attrs = []
    for _, row in edges_df.iterrows():
        a_idx = alias_to_idx.get(row['a'])
        b_idx = alias_to_idx.get(row['b'])
        if a_idx is not None and b_idx is not None:
            edges.append([a_idx, b_idx])
            edge_attrs.append(row['total_sent'])

    # Убедимся, что edge_index и edge_attr — contiguous
    edge_index = torch.tensor(edges, dtype=torch.long).T.contiguous() if edges else torch.zeros((2, 0), dtype=torch.long)
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float).view(-1, 1).contiguous() if edge_attrs else torch.zeros((0, 1), dtype=torch.float)

    # Формирование признаков узлов и меток
    x = torch.tensor(nodes_df[['degree', 'total_received']].fillna(0).values, dtype=torch.float)
    y = torch.tensor(nodes_df['label'].fillna(0).values, dtype=torch.long)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

# === Тестовый класс ===
class TestLoadGraphFromPostgres(unittest.TestCase):
    def setUp(self):
        # Конфигурация БД
        self.db_config = {
            'dbname': 'test_db',
            'user': 'test_user',
            'password': 'test_pass',
            'host': 'localhost',
            'port': 5432
        }
        # Фиктивные данные для узлов
        self.nodes_data = pd.DataFrame({
            'alias': [1, 2],
            'label': [0, 1],
            'degree': [2, 1],
            'total_received': [100.0, 200.0]
        })
        # Фиктивные данные для рёбер
        self.edges_data = pd.DataFrame({
            'a': [1, 2],
            'b': [2, 1],
            'total_sent': [50.0, 30.0]
        })

    # ✅ Тест: корректная загрузка данных
    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_load_graph_returns_correct_data(self, mock_read_sql, mock_connect):
        # Настройка мока для подключения к БД
        mock_conn = MagicMock()
        mock_cursor = MagicMock()
        mock_cursor.fetchone.return_value = [2]  # 2 узла
        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        # Настройка мока для read_sql
        def side_effect(query, *args, **kwargs):
            if 'SELECT alias' in query:
                return self.nodes_data
            elif 'SELECT a, b' in query:
                return self.edges_data
            return pd.DataFrame()

        mock_read_sql.side_effect = side_effect

        # Вызов тестируемой функции
        data = load_graph_from_postgres(self.db_config)

        # Проверка структуры Data
        self.assertIsInstance(data, Data)
        self.assertEqual(data.x.shape, (2, 2))
        self.assertEqual(data.edge_index.shape, (2, 2))
        self.assertEqual(data.edge_attr.shape, (2, 1))
        self.assertEqual(data.y.shape, (2,))

        # Проверка содержимого
        expected_x = torch.tensor([[2, 100], [1, 200]], dtype=torch.float)
        expected_edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
        expected_edge_attr = torch.tensor([[50.0], [30.0]], dtype=torch.float)
        expected_y = torch.tensor([0, 1], dtype=torch.long)

        self.assertTrue(torch.equal(data.x, expected_x))
        self.assertTrue(torch.equal(data.edge_index, expected_edge_index))
        self.assertTrue(torch.equal(data.edge_attr, expected_edge_attr))
        self.assertTrue(torch.equal(data.y, expected_y))

    # ✅ Тест: обработка отсутствия рёбер
    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_load_graph_returns_empty_edges(self, mock_read_sql, mock_connect):
        # Настройка мока
        mock_conn = MagicMock()
        mock_cursor = MagicMock()
        mock_cursor.fetchone.return_value = [2]
        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        def side_effect(query, *args, **kwargs):
            if 'SELECT alias' in query:
                return self.nodes_data
            elif 'SELECT a, b' in query:
                return pd.DataFrame(columns=['a', 'b', 'total_sent'])
            return pd.DataFrame()

        mock_read_sql.side_effect = side_effect

        data = load_graph_from_postgres(self.db_config)

        # Проверка структуры
        self.assertIsInstance(data, Data)
        self.assertEqual(data.x.shape, (2, 2))
        self.assertEqual(data.edge_index.shape, (2, 0))
        self.assertEqual(data.edge_attr.shape, (0, 1))
        self.assertEqual(data.y.shape, (2,))

        # Проверка содержимого
        expected_x = torch.tensor([[2, 100], [1, 200]], dtype=torch.float)
        expected_edge_index = torch.zeros((2, 0), dtype=torch.long)
        expected_edge_attr = torch.zeros((0, 1), dtype=torch.float)
        expected_y = torch.tensor([0, 1], dtype=torch.long)

        self.assertTrue(torch.equal(data.x, expected_x))
        self.assertTrue(torch.equal(data.edge_index, expected_edge_index))
        self.assertTrue(torch.equal(data.edge_attr, expected_edge_attr))
        self.assertTrue(torch.equal(data.y, expected_y))

    # ✅ Тест: обработка ошибок БД
    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_load_graph_handles_db_errors(self, mock_read_sql, mock_connect):
        # Настройка мока для выброса ошибки
        mock_conn = MagicMock()
        mock_cursor = MagicMock()
        mock_cursor.fetchone.side_effect = Exception("Database connection failed")
        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        # Устанавливаем side_effect для read_sql
        mock_read_sql.side_effect = Exception("Invalid SQL query")

        # Вызов функции
        data = load_graph_from_postgres(self.db_config)

        # Проверяем, что возвращается Data с пустыми тензорами
        self.assertIsInstance(data, Data)
        self.assertTrue(data.x.numel() == 0)
        self.assertTrue(data.edge_index.numel() == 0)
        self.assertTrue(data.edge_attr.numel() == 0)
        self.assertTrue(data.y.numel() == 0)

    # ✅ Тест: совместимость с NeighborLoader
    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_compatibility_with_neighbor_loader(self, mock_read_sql, mock_connect):
        # Настройка мока для подключения к БД
        mock_conn = MagicMock()
        mock_cursor = MagicMock()
        mock_cursor.fetchone.return_value = [2]  # 2 узла
        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        # Настройка мока для read_sql
        def side_effect(query, *args, **kwargs):
            if 'SELECT alias' in query:
                return self.nodes_data
            elif 'SELECT a, b' in query:
                return self.edges_data
            return pd.DataFrame()

        mock_read_sql.side_effect = side_effect

        # Вызов тестируемой функции
        data = load_graph_from_postgres(self.db_config)

        try:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", message="An issue occurred while importing 'torch-sparse'.")
                warnings.filterwarnings("ignore", message="Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated")

                from torch_geometric.loader import NeighborLoader
        except ImportError:
            self.skipTest("NeighborLoader not available (requires torch_geometric)")

        try:
            loader = NeighborLoader(
                data,
                num_neighbors=[-1],
                batch_size=2,
                input_nodes=torch.arange(data.num_nodes),
                shuffle=False  # Чтобы уменьшить неоднозначность
            )

            for batch in loader:
                self.assertIsInstance(batch, Data)

                # Проверка наличия ключевых атрибутов
                self.assertTrue(hasattr(batch, 'x'))
                self.assertTrue(hasattr(batch, 'edge_index'))
                self.assertTrue(hasattr(batch, 'edge_attr'))
                self.assertTrue(hasattr(batch, 'y'))

                # Проверка размеров
                self.assertEqual(batch.x.shape, (2, 2))
                self.assertEqual(batch.edge_index.shape[0], 2)  # 2 строки: [from, to]
                self.assertEqual(batch.edge_attr.shape, (2, 1))
                self.assertEqual(batch.y.shape, (2,))

                # Проверка значений признаков
                expected_x = torch.tensor([[2, 100], [1, 200]], dtype=torch.float)
                expected_edge_attr = torch.tensor([[50.0], [30.0]], dtype=torch.float32)
                expected_y = torch.tensor([0, 1], dtype=torch.long)

                self.assertTrue(torch.allclose(batch.x, expected_x, atol=1e-4))
                # Проверка значений edge_attr в любом порядке
                self.assertTrue(
                    torch.allclose(
                        batch.edge_attr.sort(dim=0).values,
                        expected_edge_attr.sort(dim=0).values,
                        atol=1e-3
                    )
                )
                self.assertTrue(torch.equal(batch.y, expected_y))

                # Проверка, что все индексы рёбер валидны
                self.assertLess(batch.edge_index.max(), batch.num_nodes)

                break
        except ImportError as e:
            error_msg = str(e)
            if "requires either 'pyg-lib' or 'torch-sparse'" in error_msg:
                self.skipTest("Missing optional packages: pyg-lib or torch-sparse")
            else:
                self.fail(f"Unexpected ImportError: {e}")
        except Exception as e:
            self.fail(f"NeighborLoader raised an unexpected exception: {e}")

## 3. Запуск тестов

In [3]:
# === Запуск тестов ===
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.143s

OK
